# Time model 

The court is cut into different cell. For each cell, we would like to know the closest player taking into account inertia. Mathematical development are avalaible in the file *Closest_player_to_a_point*. Here we the results of this model, particularly, how visually we can see that it takes inertia into account